# Pipeline

## Imports

In [10]:
import os
import tensorflow as tf
from preprocessing import preprocess
from dataset import make_dataset
from model import CRNN
from util import CustomDataGenerator

## Paths

In [2]:
DATA_DIR = "../../data/"
TRAIN_DIR = "train-clean-360/LibriSpeech/train-clean-360/"
VALID_DIR = "dev-clean/LibriSpeech/dev-clean/"
TEST_DIR = "test-clean/LibriSpeech/test-clean/"
NOISE_DIR = ""

PROCESSED_DIR = os.path.join(DATA_DIR, "processed/")
DATASET_DIR = os.path.join(DATA_DIR, "dataset/")
MODEL_DIR = "../models/"

## Parameters

In [3]:
initial_s = 10
s = 5
max_k = 10

## Preprocessing

In [4]:
input_dirs = [TRAIN_DIR, VALID_DIR, TEST_DIR]
datasets = ["train", "valid", "test"]

for i, dataset in enumerate(datasets):
    input_dir = os.path.join(DATA_DIR, input_dirs[i])
    output_dir = os.path.join(PROCESSED_DIR, f"{dataset}/")

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        
    preprocess(input_dir, output_dir, initial_s)

## Creating Dataset

In [5]:
for i, dataset in enumerate(datasets):
    input_dir = os.path.join(PROCESSED_DIR, f"{dataset}/")
    output_dir = os.path.join(DATASET_DIR, f"{dataset}/")

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        
    make_dataset(input_dir, output_dir, max_k)

C:\Users\Marlous\anaconda3\envs\asr\lib\site-packages\pyloudnorm\normalize.py:31: UserWarning: Possible clipped samples in output.
  warnings.warn("Possible clipped samples in output.")


## Train

### Data Generators

In [14]:
train_files = os.listdir(os.path.join(DATASET_DIR, "train/"))
valid_files = os.listdir(os.path.join(DATASET_DIR, "valid/"))

In [17]:
train_generator = CustomDataGenerator(os.path.join(DATASET_DIR, "train/"), train_files, dim=(500, 201), max_k=10, batch_size=32, shuffle=True)
valid_generator = CustomDataGenerator(os.path.join(DATASET_DIR, "valid/"), valid_files, dim=(500, 201), max_k=10, batch_size=1, shuffle=False)

### Model

In [8]:
model = CRNN((500, 201, 1), max_k)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 498, 199, 64)      640       
                                                                 
 conv2d_1 (Conv2D)           (None, 496, 197, 32)      18464     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 165, 65, 32)      0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 163, 63, 128)      36992     
                                                                 
 conv2d_3 (Conv2D)           (None, 161, 61, 64)       73792     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 53, 20, 64)       0         
 2D)                                                    

### Compile

In [12]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3, beta_1=0.9, beta_2=0.999, epsilon=1e-8),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=tf.keras.metrics.CategoricalAccuracy(),
)

### Train

In [18]:
model.fit(train_generator, validation_data=valid_generator, epochs=10)

Epoch 1/10


InvalidArgumentError: Graph execution error:

Detected at node 'categorical_crossentropy/softmax_cross_entropy_with_logits' defined at (most recent call last):
    File "C:\Users\Marlous\anaconda3\envs\asr\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\Marlous\anaconda3\envs\asr\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\Marlous\anaconda3\envs\asr\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "C:\Users\Marlous\anaconda3\envs\asr\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "C:\Users\Marlous\anaconda3\envs\asr\lib\site-packages\ipykernel\kernelapp.py", line 677, in start
      self.io_loop.start()
    File "C:\Users\Marlous\anaconda3\envs\asr\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "C: